In [ ]:
%matplotlib inline

# Segmentación con Deeplab-V3.

El propósito de este laboratorio es que el alumno se familiarice con una red neuronal convolucional (CNN) para segmentación semántica de imágenes (en la que a cada píxel de la imagen de entrada se asocia una clase de objeto). En concreto, en este tutorial se realizará _fine-tuning_ sobre la red `Deeplab-V3`. El objetivo de la práctica es que el alumno analice los bloques específicos para segmentación de la arquitectura de red, compare la segmentación binaria y la segmentación multi-clase; y trabaje sobre otros conceptos como el _data augmentation_ o la función de coste de la red neuronal en un escenario real con el objetivo de mejorar el aprendizaje de la misma. 

En concreto, se realizará _fine-tuning_ sobre la red `Deeplab-V3`, una de las redes del estado del arte para segmentación semántica. En nuestro caso, la tarea de segmentación a la que se va a aplicar esta arquitectura es la segmentación del disco óptico y su parte central (la cúpula óptica) en retinografía.

Este tutorial es una adaptación del tutorial disponible en https://expoundai.wordpress.com/2019/08/30/transfer-learning-for-segmentation-using-deeplabv3-in-pytorch/.

Referencias:
- [1] Deeplab-V3. https://arxiv.org/abs/1706.05587
- [2] Imagen. https://raw.githubusercontent.com/abreheret/PixelAnnotationTool/master/images_test/Abbey_Road.jpg
- [3] Imagen. https://raw.githubusercontent.com/abreheret/PixelAnnotationTool/master/images_test/Abbey_Road_color_mask.png
- [4] Deeplab. https://arxiv.org/abs/1606.00915
- [5] RIGA dataset. http://academictorrents.com/details/eb9dd9216a1c9a622250ad70a400204e7531196d
- [6] Regularización local. http://www.sfu.ca/~abentaie/papers/miccai16.pdf
- [7] Focal Loss for Object Detection. https://arxiv.org/abs/1708.
- [8] Dice Loss. https://arxiv.org/abs/1606.04797

## Antes de empezar

Antes de empezar, necesita configurar algunas cosas en caso de que vaya a utilizar Google Colab. En particular, necesita descomprimir los archivos de la práctica en una carpeta en Drive y cambiar el directorio de trabajo al de dicha carpeta. Para ello, ejecute el siguiente código:

In [ ]:
#Descomenta únicamente si quieres ejecutar este código en Google Colab
#from google.colab import drive
#import os, sys
#drive.mount('/content/drive')
#print(os.getcwd())
#os.chdir('/content/drive/My Drive/Colab Notebooks/segmentation_folder') #Here put the full path to the folder where you have the files

Además, si quiere ejecutar el código con soporte a GPU, en Google Colab vaya a `Entorno de ejecución->Cambiar tipo entorno de ejecución` y seleccione GPU en `acelerador por hardware`.

## Parte 1. Fundamento teórico.

### Segmentación de objetos

Frente a los métodos tradicionales de segmentación de imágenes (*thresholding, clustering* o *region growing*), las técnicas de *deep learning* han demostrado ser mucho más eficaces para tareas complejas de segmentación de imágenes. Sin embargo, estas técnicas requieren grandes bases de datos anotadas píxel a píxel para su entrenamiento, lo que supone un gran esfuerzo de anotación. 
 
El objetivo de un algoritmo de segmentación de objetos es generar máscaras de salida a nivel de píxel en las cuales a las regiones que pertenecen a ciertas categorías se les asigna el mismo valor de píxel. Si se codifican en color (asignando un color diferente a cada clase de objetos) se obtienen resultados como los que se muestran en la siguiente figura [2-3], donde en azul se representa la clase vehículo, en rojo la clase persona, etc.

<table><tr><td><img src="http://tsc.uc3m.es/~mmolina/images_segmentation/beatles.jpg"></td><td><img src="http://tsc.uc3m.es/~mmolina/images_segmentation/beatles_mask.png"></td></tr></table>

Por tanto, como entrada a nuestro algoritmo de segmentación se tendrá un conjunto de imágenes y sus correspondientes máscaras *ground truth* píxel a píxel.

### Contexto vs resolución

El principal reto en segmentación semántica es encontrar un compromiso entre la importancia del contexto global de la imagen (para segmentar un objeto, es necesario identificar sus distintas partes y diferenciarlo del resto de objetos de la imagen), y las características locales de la imagen (una segmentación precisa ha de analizar los valores de los píxeles que se encuentran en los alrededores del objeto para delimitar la frontera del mismo). La siguiente figura muestra las distintas estrategias que se utilizan para intentar representar este compromiso sobre CNNs.

<img src="http://tsc.uc3m.es/~mmolina/images_segmentation/arquitecturas.PNG">

La aproximación tradicional que se ha seguido para utilizar extraer características de alto nivel de la imagen ha sido el análisis de la misma a través de una representación en pirámide, en la que se consideran la imagen original y versiones de menor tamaño de la misma (filtrado gaussiano y reducción del tamaño mediante submuestreo). Si se aplica el mismo procesado a todos los niveles de la pirámide, se obtienen características de bajo nivel en los primeros niveles, y de alto nivel conforme se va profundizando en la misma (véase la figura a) arriba, donde se puede ver que se mezclan las características de dos pirámides de imagen que parten de distintas escalas). De esta manera, en las versiones de menor tamaño pierde importancia la información local de la imagen en favor de la información global, y viceversa. Las arquitecturas de redes neuronales convolucionales (FCNs) de clasificación se basan en esta representación para intentar resumir la información de la imagen (las características a niveles menos profundos sirven para calcular características a niveles más profundos) y proporcionar su categoría. 

Por el contrario, en segmentación no solo importa el contexto global de la imagen, sino que las características locales también cuentan. Las redes totalmente convolucionales para segmentación (*Fully Convolutional Networks*, FCNs) introducen una estructura *encoder-decoder* en la red para obtener una salida con una resolución igual (o lo más cercana posible) a la de la entrada (véase la figura b) arriba). Para ello, el *encoder* reduce paulatinamente la dimensión de los mapas de características de manera que la información global se captura en las capas profundas; y el *decoder* parte de esta información global y paulatinamente recupera la dimensión original de la imagen. Frecuentemente se incluyen conexiones entre las capas de igual dimensión del *encoder* y el *decoder* para facilitar el empleo directo de las características más locales en la salida final de la segmentación.

En las redes Deeplab [1]-[4] se hace uso de las dos estrategias de la derecha: convoluciones *atrous* y *Spatial Pyramid Pooling*. Estas estrategias se explican en la siguiente sección.


### Deeplab-V3

Aunque se recomienda echar un vistazo al artículo sobre Deeplab-V3 [1], en esta sección se van a explicar los conceptos más importantes de la red que son necesarios para el desarrollo de la práctica.

Deeplab-V3 es un _framework_ que adapta cualquier red convolucional dedicada a la clasificación (dada una imagen, encontrar una categoría que describa el contenido total de la imagen) a la segmentación de objetos. Para ello, a partir de un _backbone_ inicial (las capas destinadas a la extracción de características) de cualquier red de clasificación, propone una serie de capas y bloques destinados a extraer características de contexto en la imagen sin comprometer la resolución de los mapas de características (sin reducir aun más su resolución). Para ello, hace uso de las dos estrategias que se describen a continuación.

#### Convoluciones *atrous*

La contribución principal de la familia de redes Deeplab [4] es el diseño de las convoluciones *atrous* o convoluciones *dilated*. Este tipo de convoluciones se utilizan para reemplazar la estrategia tradicional de reducir el tamaño de la imagen a través de capas de *max pooling* con *stride* a lo largo de la red para obtener representaciones más globales del contenido de la imagen. Esta estrategia estándar hace que el *stride* acumulado a lo largo de la red sea muy elevado (32, por ejemplo, lo que significa que el tamaño de las características de una imagen original de $HxW$ es $H/32xW/32$), lo que puede ser contraproducente en segmentación (a pesar de obtener mejores características de contexto, la reducción de las dimensiones de la imagen hace que las segmentaciones sean menos precisas). Las convoluciones *atrous*, por su parte, mantienen la resolución de la entrada a la vez que extraen características de mayor orden a través del uso del *stride*. Es decir, para computar el valor de un cierto píxel $y[i]$ se toman los valores de los píxeles alejados del mismo $r$ posiciones en la entrada $x$ multiplicados por el elemento correspondiente del filtro $w$.

\begin{equation}y[i]=\sum_{k}{x[i+r\cdot k] w[k]}.\end{equation}

En modo filtro, una convolución *atrous* con tasa $r$ se consigue añadiendo $r-1$ ceros entre los elementos del filtro original, como se muestra en la siguiente figura [1]:

<img src="http://tsc.uc3m.es/~mmolina/images_segmentation/atrous.PNG" width="600pix">

A continuación se puede observar una animación en la que se comparan una convolución 2D tradicional con una convolución *atrous*, ambas con el mismo *field of view*, pero la primera usa 15 parámetros y la segunda 9 parámetros. Mientras que la convolución estándar usa un filtro de $5x5$, *stride* de 1, *dilate* de 1 y *padding* de 1; la convolución *atrous* usa un filtro de $3x3$, *stride* $r=2$, *dilate* de 2 y no usa *padding*. 

<table><tr><td><img src="http://tsc.uc3m.es/~mmolina/images_segmentation/2dconv.gif"></td><td><img src="http://tsc.uc3m.es/~mmolina/images_segmentation/atrousconv.gif"></td></tr></table>

En la práctica, estas convoluciones no se utilizan a lo largo de toda la red: en primer lugar, porque reducir el tamaño de la imagen en las primeras capas de la red es útil desde el punto de vista del coste computacional (sería muy costoso trabajar con las imágenes a tamaño completo durante todo el procesado); y en segundo lugar, porque de hecho esta sustitución del *max pooling* por convoluciones *atrous* es más útil en capas profundas de la red (con menos tasa $r$ se recorren grandes porciones de imagen). Un ejemplo de flujo de trabajo sobre ResNet se muestra a continuación [1]:

<img src="http://tsc.uc3m.es/~mmolina/images_segmentation/atrous2.PNG">

Se puede ver como a partir del bloque 3 se utilizan convoluciones *atrous* __en cascada__ con tasa variable $2^n$ para sustituir al *max pooling* con tasa 2. De este modo se mantiene la resolución en valores aceptables. 

#### *Atrous Spatial Pyramid Pooling*

La segunda contribución del artículo [1] tiene que ver con el uso de las convoluciones *atrous* como extractores de características __en paralelo__ con distinta tasa $r$, de manera que las características que se extraen en cada rama sean más globales o locales en función de dicha tasa. Esto es lo que se llama ASPP (*Atrous Spatial Pyramid Pooling*). Las características de cada rama se pueden agrupar bien mediante suma o bien mediante concatenación, para obtener después la salida final de segmentación de la red.

En concreto, la estructura del *frawework* Deeplab-V3 es la que se muestra a continuación:

<img src="http://tsc.uc3m.es/~mmolina/images_segmentation/aspp.PNG">

Se puede observar que se parte de un mapa de características de la imagen con *stride* 16 al que se añade una capa con convoluciones *atrous* con tasa $r=2$ y posteriormente el bloque ASPP. Este consta de los siguientes módulos:

- Una capa con una convolución 1x1 que extrae características más locales del mapa de entrada.
- Sendas convoluciones *atrous* con tasas $r=6$, $r=12$ y $r=18$, respectivamente, que extraen características con diferente contexto global de los mapas de entrada.
- Un *avg pooling* a nivel de mapa de entrada que genera como salida la media de cada canal. Esta característica global de la imagen permite ponderar la importancia de los distintos canales.

### Medidas de evaluación 

Para evaluar la calidad en la segmentación de objetos, se suele utilizar la medida *Intersection over Union*, IoU, también denominada *Jaccard Index* (JI). La medida $IoU$ mide la similitud entre dos regiones $A$ y $B$ como:

\begin{equation}
IoU=\frac{A \cap B}{A \cup B}
\end{equation}

siendo $\cap$ la intersección entre las regiones (area común) y $\cup$ la unión o área total que cubren entre ambas. Se considera un umbral mínimo de IoU en torno a $IoU_{th}=0.7$ para considerar una detección como correcta. A continuación se puede ver un ejemplo de la medida $IoU$.

<img src="http://tsc.uc3m.es/~mmolina/images_segmentation/iou.png">


### Utilidad de la tarea y base de datos

El glaucoma es una de las mayores causas de ceguera irreversible en el mundo. Los modelos actuales de atención médica no son capaces de acortar distancias entre la prevalencia progresiva del glaucoma y los retos para el acceso a la atención médica. La tele-oftalmología y los sistemas de ayuda al diagnóstico basados en *deep-learning* pueden ayudar a acortar esta distancia. En concreto, la retinografía o fotografía del fondo de ojo es la mejor técnica para el estudio de la papila óptica, con el objetivo detectar y evaluar síntomas del glaucoma. Los estudios que se realizan se basan en las cinco reglas siguientes:

1. Observar el anillo escleral para identificar los límites del disco óptico y su tamaño.
2. Identificar el tamaño del anillo.
3. Examinar la capa de fibras del nervio óptico.
4. Examinar por fuera la región del disco óptico en busca de atrofia parapapilar.
5. Observar si hay hemorragias retinales o del disco óptico.

Para estas tareas es fundamental localizar de manera precisa el disco óptico y la cúpula óptica.

El disco óptico, papila óptica o punto ciego es una zona circular situada en el centro de la retina, por donde salen del ojo los axones de las células ganglionares de la retina que forman el nervio óptico. Esta área mide 1.5 x 2.5 mm en el ojo humano y carece de sensibilidad a los estímulos luminosos por no poseer ni conos ni bastones, ello causa una zona ciega dentro del campo visual que se conoce como punto ciego. Dentro de la papila se encuentra una excavación fisiológica llamada cúpula, en el centro de la misma. La siguiente figura, a la izquierda, muestra una animación con el disco óptico en color rosado y la cúpula óptica en color blanco. A la derecha, se puede ver un ejemplo de anotación del disco óptico y la cúpula óptica. 

<table><tr><td><img src="http://tsc.uc3m.es/~mmolina/images_segmentation/animation.gif" width="500pix"></td><td><img src="http://tsc.uc3m.es/~mmolina/images_segmentation/annotation.jpg" width="500pix"></td></tr></table>

El objetivo de esta práctica será segmentar el disco y la cúpula ópticas en retinografía. En un entorno real esto es un paso previo a la extracción de características: por ejemplo, el cociente entre el diámetro de la cúpula y el diámetro del disco óptico es un indicador del daño que origina el glaucoma; y el diagnóstico.

La base de datos de la que se dispone corresponde a una versión procesada de la base de datos disponible en [5]. Esta base de datos contiene 698 imágenes de retinografía procedentes de 3 proyectos diferentes: MESSIDOR, Magrabia y BinRushed. Las anotaciones de la cúpula y el disco ópticos con las que se cuenta para cada imagen se han obtenido a partir de la concordancia entre las anotaciones de 6 oftalmólogos expertos para cada caso. 

Los conjuntos se distribuyen de la siguiente manera:
- Entrenamiento: 400 imágenes con su anotación correspondiente.
- Validación: 148 imágenes con su anotación correspondiente.
- Test: 150 imágenes con su anotación correspondiente.

## Parte 2. Implementación.

En primer lugar, se importan las librerías necesarias y se definen algunos parámetros generales.

In [ ]:
import os
import glob
from tqdm import tqdm
import numpy as np
import copy
import time
import torch
import torch.nn.functional as FT
from torch.utils.data import Dataset, DataLoader
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
from torchvision.models.segmentation import deeplabv3_resnet101
from torchvision import transforms, utils
import torchvision.transforms.functional as F
from PIL import Image
import cv2
import csv
import random
import matplotlib.pyplot as plt
# Set random seed for reproducibility
manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
# torch uses some non-deterministic algorithms
torch.backends.cudnn.enabled = False

### Entradas

Se definen algunas entradas para la ejecución:

-  **data_dir** - el directorio raíz de la base de datos, que se describe posteriormente.
-  **num_classes_binary** - el número de clases que detectar en el problema binario.
-  **class_names_binary** - los nombres de las clases que detectar en el problema binario.
-  **num_classes_multiclass** - el número de clases que detectar en el problema multi-clase.
-  **class_names_multiclass** - los nombres de las clases que detectar en el problema multi-clase.
-  **img_size** - el tamaño de las imágenes de entrada a la red (cuadradas).
-  **batchsize_train** - el tamaño de _batch_ que se utiliza para entrenamiento.
-  **batchsize_test** - el tamaño de _batch_ que se utiliza para test.
-  **epochs** - número de _epochs_ para el entrenamiento de la red.
-  **step_size** - número de *epochs* tras los cuales se reduce el *learning rate* en un factor 0.1.
-  **result_dir** - el directorio raíz para almacenar los resultados.
-  **device** - el dispositivo (GPU o CPU) para la ejecución.

In [ ]:
data_dir = 'riga'
num_classes_binary=2 # First, we use Deeplab-V3 for binary segmentation
class_names_binary=['background','optic_cup']
num_classes_multiclass=3 
class_names_multiclass=['background', 'optic_cup', 'optic_disk']
img_size=512           # Size of the images
batchsize_train=2      # Batch size for training
batchsize_test=1       # Batch size for test (it must be one to generate predictions)
epochs = 8             # Number of epochs to train (must be pair)
step_size=5
result_dir = 'results' # Result directory
# Use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### Red

La red a utilizar será una versión de Deeplab-V3 con ResNet-101 como *backbone*, ya pre-entrenada sobre el conjunto de entrenamiento de la base de datos COCO. En concreto, se utilizan los 4 primeros bloques de esta red, compuestos de capas convolucionales, normalización de batch (cuyos parámetros se congelan) y capas no lineales. Vea como los bloques se modifican añadiendo convoluciones *atrous* en lugar de *max pooling*, como se ha descrito en el apartado teórico, y a partir del cuarto bloque se añade el bloque de ASPP y la convolución que genera la salida de segmentación.

A continuación se incluye la función que construye la red. Analice la arquitectura.

- ¿Se realiza el submuestreo con capas de *max pooling* en las capas iniciales, y *layers* 1, 2 y 3? ¿Por qué?
- ¿Coinciden las tasas $r$ de las convoluciones *atrous* de los bloques 4 y el ASPP con el apartado teórico? ¿Por qué? ¿Cuál es el stride acumulado hasta la capa 4?
- ¿Cuál es la diferencia en cuanto a los parámetros entre una convolución *atrous* (mantiene el tamaño de la imagen) y las convolución que sustituyen al *max pooling*?

__**IMPORTANTE:__ no preste atención al `aux_classifier` de momento. Se trata de un clasificador auxiliar que se utiliza para mejorar el problema del *vanishing gradient*. Si se introduce una función de pérdida a su salida y se realiza la retropropagación, este bloque introduce gradientes más robustos en un punto intermedio de la red que pueden ayudar al entrenamiento (dado que la arquitectura es muy profunda, los gradientes de la salida estándar se van desvaneciendo a medida que se va avanzando desde la salida del clasificador estándar hasta el inicio de la red). Por el momento no se va a utilizar. 

In [ ]:
def get_deeplabv3(num_classes=1):
    model = deeplabv3_resnet101(pretrained=True, progress=True)
    model.classifier = DeepLabHead(2048, num_classes)
    return model

In [ ]:
model = get_deeplabv3(num_classes_binary)
print(model)
model.to(device)

__El modelo no es el mismo estudiado en el apartado teórico. En concreto hay una convolución con stride y max-pooling en el bloque 0 (rápido se reduce el tamaño en 4), en la layer 1 no existe downsample, en la layer 2 sí y en la layer 3 no, de manera que el stride acumulado hasta el bloque 4 es 8. Por ello, el bloque 4 contiene dos convoluciones *atrous*, una con $r=2$ y otra con $r=4$ (para aumentar el *receptive field* a la entrada del ASPP). Además, en el ASPP se consideran tasas $r=12, 24, 36$, el doble de las del apartado teórico__.

### Base de datos

En esta práctica se va a realizar tanto segmentación binaria como segmentación multi-clase. La base de datos se proporciona en forma de imágenes de retinografía y máscaras de objetos. Se proporcionan dos tipos de máscara: las carpetas `masks`, que contienen máscaras donde solo aparece la cúpula óptica (clase 1), que utilizaremos durante la primera parte de la práctica; y las carpetas `masks_full` donde aparecen la cúpula óptica (clase 1) y el disco óptico (clase 2). 

- Segmentación binaria: las máscaras que se proporcionan en este caso suelen ser imágenes `uint8` con el valor 0 para la clase `background` y 255 para la clase `optic_cup`.

- Segmentación multi-clase: hay que modificar la clase de carga de red y la normalización de los datos (en este caso cada clase corresponde a un entero distinto en la máscara): 0 para la clase `background`, 1 para la clase `optic_cup` y 2 para la clase `optic_disk`.

A continuación se define la clase que implementa la carga de la base de datos para ambos casos. Preste atención a las opciones `maskFolder` y `binary` para escoger entre una y otra.

__**IMPORTANTE:__ para acelerar los experimentos si fuera necesario, se proporciona código para seleccionar el número de muestras con las que se entrena la red.

In [ ]:
class RigaDataset(Dataset):
    """Binary Segmentation Dataset"""

    def __init__(self, root_dir, imageFolder, maskFolder, binary=True, use_only_train=[], transform=None, subset=None,  imagecolormode='rgb', maskcolormode='grayscale'):
        """
        Args:
            root_dir (string): Directory with all the images and should have the following structure.
            root
            --Images
            -----Img 1
            -----Img N
            --Mask
            -----Mask 1
            -----Mask N
            imageFolder (string) = 'Images' : Name of the folder which contains the Images.
            maskFolder (string)  = 'Masks : Name of the folder which contains the Masks.
            transform (callable, optional): Optional transform to be applied on a sample.
            subset: 'Train', 'Val' or 'Test' to select the appropriate set.
            imagecolormode: 'rgb' or 'grayscale'
            maskcolormode: 'rgb' or 'grayscale'
        """
        self.color_dict = {'rgb': 1, 'grayscale': 0}
        assert(imagecolormode in ['rgb', 'grayscale'])
        assert(maskcolormode in ['rgb', 'grayscale'])

        self.imagecolorflag = self.color_dict[imagecolormode]
        self.maskcolorflag = self.color_dict[maskcolormode]
        self.root_dir = root_dir
        self.transform = transform
        self.binary=binary
        assert(subset in ['Train', 'Val', 'Test'])
        if (subset=='Train'):
            self.image_names = sorted(glob.glob(os.path.join(self.root_dir, 'train', imageFolder, '*')))
            self.mask_names = sorted(glob.glob(os.path.join(self.root_dir, 'train', maskFolder, '*')))
            if (len(use_only_train)>1):
                self.image_names=[self.image_names[x] for x in use_only_train]
                self.mask_names=[self.mask_names[x] for x in use_only_train]
        elif(subset=='Val'):
            self.image_names = sorted(glob.glob(os.path.join(self.root_dir, 'val', imageFolder, '*')))
            self.mask_names = sorted(glob.glob(os.path.join(self.root_dir, 'val', maskFolder, '*')))
        else:
            self.image_names = sorted(glob.glob(os.path.join(self.root_dir, 'test', imageFolder, '*')))
            self.mask_names = sorted(glob.glob(os.path.join(self.root_dir, 'test', maskFolder, '*')))
            
    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        if self.imagecolorflag:
            image = cv2.imread(
                img_name, self.imagecolorflag).transpose(2, 0, 1)
        else:
            image = cv2.imread(img_name, self.imagecolorflag)
        msk_name = self.mask_names[idx]
        if self.maskcolorflag:
            mask = cv2.imread(msk_name, self.maskcolorflag).transpose(2, 0, 1)
        else:
            mask = cv2.imread(msk_name, self.maskcolorflag)
            if (self.binary):
                # In a binary dataset, the masks are usually given in uint8 form: 0-background, 255 foreground
                # For the BCELoss, we need the background mask (inverse of the foreground one)
                mask2 = 255-mask
                # We concatenate both
                mask = np.concatenate((mask2[np.newaxis,:,:],mask[np.newaxis,:,:]),axis=0)
        sample = {'image': image, 'mask': mask, 'img_path': img_name}

        if self.transform:
            sample = self.transform(sample)

        return sample

# Define few transformations for the Segmentation Dataloader
class Augm(object):
    """Perform data augmentation. NOT IMPLEMENTED YET."""
    def __init__(self, data_augm):
        self.data_augm = data_augm
    def __call__(self, sample):
        img, mask, img_path = sample['image'], sample['mask'], sample['img_path']
        if (self.data_augm):
            # DATA AUGMENTATION: IMPLEMENT YOUR VERSION
            pass
        else:
            pass
        
        return {'image': img,
                'mask': mask, 
                'img_path': img_path}

class Resize(object):
    """Resize image and/or masks."""

    def __init__(self, imageresize, maskresize):
        self.imageresize = imageresize
        self.maskresize = maskresize

    def __call__(self, sample):
        image, mask, img_path = sample['image'], sample['mask'], sample['img_path']
        if len(image.shape) == 3:
            image = image.transpose(1, 2, 0)
        if len(mask.shape) == 3:
            mask = mask.transpose(1, 2, 0)
        mask = cv2.resize(mask, self.maskresize, interpolation=cv2.INTER_NEAREST)
        image = cv2.resize(image, self.imageresize, interpolation=cv2.INTER_AREA)
        if len(image.shape) == 3:
            image = image.transpose(2, 0, 1)
        if len(mask.shape) == 3:
            mask = mask.transpose(2, 0, 1)

        return {'image': image,
                'mask': mask,
                'img_path': img_path}

class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample, maskresize=None, imageresize=None):
        image, mask, img_path = sample['image'], sample['mask'], sample['img_path']
        if len(image.shape) == 2:
            image = image.reshape((1,)+image.shape)
        return {'image': torch.from_numpy(image),
                'mask': torch.from_numpy(mask), 
                'img_path': img_path}


class NormalizeBinary(object):
    '''Normalize image'''
    def __init__(self, image_mean,image_std):
        self.image_mean = image_mean
        self.image_std = image_std
    def __call__(self, sample):
        image, mask, img_path = sample['image'], sample['mask'], sample['img_path']
        # Binary segmentation: masks can be float with 0-1 values
        if (self.image_mean is None):
            return {'image': image.type(torch.FloatTensor)/255,
                    'mask': mask.type(torch.FloatTensor)/255, 
                    'img_path': img_path}
        else:   
            return {'image': F.normalize(image.type(torch.FloatTensor)/255,self.image_mean,self.image_std),
                    'mask': mask.type(torch.FloatTensor)/255, 
                    'img_path': img_path}

        
class NormalizeMulticlass(object):
    '''Normalize image'''
    def __init__(self, image_mean,image_std):
        self.image_mean = image_mean
        self.image_std = image_std
    def __call__(self, sample):
        image, mask, img_path = sample['image'], sample['mask'], sample['img_path']
        if (self.image_mean is None):
            # Masks must be LongTensor with 0-1-2... indicators for the classes
            return {'image': image.type(torch.FloatTensor)/255,
                    'mask': mask.type(torch.LongTensor), 
                    'img_path': img_path}
        else:   
            return {'image': F.normalize(image.type(torch.FloatTensor)/255,self.image_mean,self.image_std),
                    'mask': mask.type(torch.LongTensor), 
                    'img_path': img_path}

def get_dataloader_riga(data_dir, imageFolder='images', maskFolder='masks', binary=True, use_only_train=[], batch_size=4, img_size=256, data_augm=False, image_mean=None,image_std=None):
    """
        Create training, validation and testing dataloaders from a single folder.
    """
    if (binary):
        data_transforms = {
            'Train': transforms.Compose([Augm(data_augm), Resize((img_size,img_size),(img_size,img_size)), ToTensor(), NormalizeBinary(image_mean,image_std)]),
            'Val': transforms.Compose([Resize((img_size,img_size),(img_size,img_size)), ToTensor(), NormalizeBinary(image_mean,image_std)]),
            'Test': transforms.Compose([Resize((img_size,img_size),(img_size,img_size)), ToTensor(), NormalizeBinary(image_mean,image_std)]),
            }
        image_datasets = {x: RigaDataset(data_dir, imageFolder=imageFolder, maskFolder=maskFolder, binary=binary, use_only_train=use_only_train, subset=x, transform=data_transforms[x])
                          for x in ['Train', 'Val', 'Test']}
        
    else:
        data_transforms = {
            'Train': transforms.Compose([Augm(data_augm), Resize((img_size,img_size),(img_size,img_size)), ToTensor(), NormalizeMulticlass(image_mean,image_std)]),
            'Val': transforms.Compose([Resize((img_size,img_size),(img_size,img_size)), ToTensor(), NormalizeMulticlass(image_mean,image_std)]),
            'Test': transforms.Compose([Resize((img_size,img_size),(img_size,img_size)), ToTensor(), NormalizeMulticlass(image_mean,image_std)]),
            }
        image_datasets = {x: RigaDataset(data_dir, imageFolder=imageFolder, maskFolder=maskFolder, binary=binary, use_only_train=use_only_train, subset=x, transform=data_transforms[x])
                          for x in ['Train', 'Val', 'Test']}

    dataloaders = {'Train': DataLoader(image_datasets['Train'], batch_size=batch_size,
                                 shuffle=True, num_workers=8),
                   'Val': DataLoader(image_datasets['Val'], batch_size=batch_size,
                                 shuffle=False, num_workers=8),
                   'Test': DataLoader(image_datasets['Test'], batch_size=batch_size,
                                 shuffle=False, num_workers=8)}
    return dataloaders


Se carga la base de datos, ya que en la primera parte de la práctica trabajaremos con segmentación binaria.

In [ ]:
# Use the generic mean if it is not given
if (not os.path.exists(os.path.join(data_dir,'mean-channel.npy'))):
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
else:
    mean=np.load(os.path.join(data_dir,'mean-channel.npy'))
    std=np.load(os.path.join(data_dir,'std-channel.npy'))
    
# Code to use only XX images of the database for train
#n_train=XX
#np.random.seed(manualSeed)
#ids=np.random.permutation(400)
#use_only_train=ids[:n_train] #
use_only_train=[] # if we want to use all images
dataloaders = get_dataloader_riga(
    data_dir, maskFolder='masks', binary=True, use_only_train=use_only_train, batch_size=batchsize_train, img_size=img_size, data_augm=False, image_mean=mean,image_std=std)

### Medidas de evaluación

Asimismo, se define la función que testea la bondad de nuestro modelo de segmentación binario en términos de segmentación (índice Jaccard o *Intersection over Union*).

La función recibe como parámetros:

- __model__: la CNN que evaluar.
- __dataloader__: el cargador de los datos de test.
- __class_names__: nombres de las clases de objetos a detectar (siempre hay que incluir en primer lugar la clase *background*).
- __TH__: la red proporciona un *score* que convertimos a probabilidad *softmax* para cada píxel. Este umbral distingue las dos clases.
- __result_dir__: el directorio donde guardar los resultados.
- __SAVE_OPT__: para guardar o no los resultados de test como imágenes con el *ground truth* en verde y la segmentación en rojo, en el directorio `predictions_binary`.
- __batchsize__: debe ser igual a 1.


In [ ]:
def test_segmentation_model_binary(model, dataloaders, class_names, TH, result_dir, SAVE_OPT, batch_size=1):
    # Evaluation: Jaccard Index
    jaccard=[]
    # We create the results folder and CSV file for results if they do not exist
    if not os.path.exists(result_dir):
        os.mkdir(result_dir)
    else:
        if os.path.exists(os.path.join(result_dir,'results_binary.csv')):
            os.remove(os.path.join(result_dir,'results_binary.csv'))
    if (SAVE_OPT):
        # We create the folder for predictions
        if not os.path.exists(os.path.join(result_dir,'predictions_binary')):
            os.mkdir(os.path.join(result_dir,'predictions_binary'))
    csv_file=open(os.path.join(result_dir,'results_binary.csv'),'w')
    coord_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for sample in dataloaders['Test']:
        with torch.no_grad():
            inputs = sample['image'].to(device)
            masks = sample['mask'].to(device)
            prediction = model(inputs)
            y_pred = prediction['out']
            y_pred = torch.nn.functional.softmax(y_pred, dim=1)
            y_pred = y_pred[:,1,:,:].data.cpu().numpy()
            y_true = masks.data.cpu().numpy()
            y_true = y_true[:,1,:,:]
            for j in range(y_pred.shape[0]):
                img_name=sample['img_path'][j].split(os.path.sep)[-1]
                dataset_name=sample['img_path'][j].split(os.path.sep)[-2]
                # We measure with the Jaccard Index
                ji=np.sum(np.logical_and(np.squeeze(y_pred[j,:,:]>TH),np.squeeze(y_true[j,:,:]>0)))/(np.sum(y_pred[j,:,:]>TH)+np.sum(y_true[j,:,:]>0)-np.sum(np.logical_and(np.squeeze(y_pred[j,:,:]>TH),np.squeeze(y_true[j,:,:]>0))))
                jaccard.append(ji)
                if (SAVE_OPT):
                    img=Image.open(sample['img_path'][j]).resize((img_size,img_size))
                    mask=(np.transpose(np.concatenate(((y_pred[j,:,:]>TH)[np.newaxis,:,:],y_true[j,:,:][np.newaxis,:,:],np.zeros_like(y_pred[j,:,:][np.newaxis,:,:])),axis=0),(1,2,0))*255.0).astype(np.uint8)
                    img = Image.blend(img.convert('RGBA'), Image.fromarray(mask).convert('RGBA'),0.5)
                    img.save(os.path.join(result_dir,'predictions_binary',img_name[:-4]+'.png'))
                coord_writer.writerow([img_name,str(ji)])
    # Mean values
    coord_writer.writerow(['MEAN',str(sum(jaccard)/len(jaccard))])
    print('Jaccard index for segmentation: {}'.format(sum(jaccard)/len(jaccard)))
    csv_file.close()


### Entrenamiento


La función recibe como parámetros:

- __model__: la CNN que evaluar.
- __dataloader__: el cargador de los datos de test.
- __device__: el dispositivo que utilizar para el entrenamiento (GPU o CPU).
- __optimizer__: el optimizador.
- __lr_scheduler__: la política de modificación de la tasa de aprendizaje.
- __metrics__: las métricas para medir el rendimiento del sistema de segmentación.
- __bpath__: el directorio donde almacenar la mejor de las redes de segmentación.
- __num_classes__: el número de clases para la segmentación.
- __num_epochs__: el número de *epochs* durante los que entrenar.

__**IMPORTANTE:__ normalmente, cuando se realiza un procedimiento de _fine-tuning_ sobre una red ya pre-entrenada, la base de datos de que se dispone es pequeña y no se dispone de gran capacidad de computación (que permitiría usar *batches* más grandes). En estos casos es una buena práctica fijar los módulos de normalización de *batch* de la red (poniéndolos en modo `eval`).

In [ ]:
from sklearn.metrics import roc_auc_score, jaccard_score
from sklearn.preprocessing import LabelBinarizer


# We set the batchnorm modules to eval
def set_bn_eval(mm):
    if isinstance(mm, torch.nn.modules.batchnorm._BatchNorm):
        mm.eval()
        
def train_model(model, criterion, dataloaders, device, optimizer, lr_scheduler, metrics, bpath, model_name, num_classes=2, num_epochs=3):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_jaccard = 0
    # Initialize the log file for training and testing loss and metrics
    fieldnames = ['epoch', 'Train_loss', 'Val_loss'] + \
        [f'Train_{m}' for m in metrics.keys()] + \
        [f'Val_{m}' for m in metrics.keys()]
    with open(os.path.join(bpath, 'log.csv'), 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

    for epoch in range(1, num_epochs+1):
        if os.path.exists(os.path.join(bpath,model_name+'-epoch{}.pth'.format(epoch))):
            print("=> loading checkpoint '{}'".format(epoch))
            checkpoint = torch.load(os.path.join(bpath,model_name+'-epoch{}.pth'.format(epoch)))
            lr_scheduler.load_state_dict(checkpoint['scheduler'])
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            best_jaccard=checkpoint['best_jaccard']
            print("=> loaded checkpoint '{}'" .format(epoch))
        else:
            print('Epoch {}/{}'.format(epoch, num_epochs))
            print('-' * 10)
            # Each epoch has a training and validation phase
            # Initialize batch summary
            batchsummary = {a: [0] for a in fieldnames}

            for phase in ['Train', 'Val']:
                if phase == 'Train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode
                model.apply(set_bn_eval)

                # Iterate over data.
                for sample in tqdm(iter(dataloaders[phase])):
                    inputs = sample['image'].to(device)
                    masks = sample['mask'].to(device)
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'Train'):
                        outputs = model(inputs)
                        loss = criterion(outputs['out'], masks)
                        
                        y_pred = outputs['out']#.data.cpu().numpy().ravel()
                        y_prob = torch.nn.functional.softmax(y_pred, dim=1)
                        _,y_pred = torch.max(y_prob,dim=1)
                        y_pred = y_pred.data.cpu().numpy().ravel()
                        y_prob = np.reshape(np.transpose(y_prob.data.cpu().numpy(),(1,0,2,3)),(num_classes,y_pred.shape[0]))
                        y_true = masks.data.cpu().numpy()#.ravel()
                        if (num_classes==2):
                            y_true = y_true>0
                        lb = LabelBinarizer()
                        lb.fit([f for f in range(0,num_classes)])
                        for name, metric in metrics.items():
                            if name == 'jaccard_score':
                                if (num_classes==2):
                                    ji=metric(y_true[:,1,:,:].reshape(-1,1), y_pred, average=None)
                                else:
                                    ji=metric(y_true.reshape(-1,1), y_pred, labels=np.unique(y_true),average=None)
                                batchsummary[f'{phase}_{name}'].append(
                                    np.mean(ji[1:]))
                            else:
                                if (num_classes==2):
                                    batchsummary[f'{phase}_{name}'].append(
                                        metric(y_true[:,1,:,:].reshape(-1,1), y_prob[1:,:].T, average='micro',multi_class='ovr'))
                                else:
                                    batchsummary[f'{phase}_{name}'].append(
                                        metric(lb.transform(y_true.reshape(-1,1)), y_prob.T, average='micro',multi_class='ovr'))

                        # backward + optimize only if in training phase
                        if phase == 'Train':
                            loss.backward()
                            optimizer.step()
                batchsummary['epoch'] = epoch
                epoch_loss = loss
                batchsummary[f'{phase}_loss'] = epoch_loss.item()
                print('{} Loss: {:.4f}'.format(
                    phase, loss))
            for field in fieldnames[3:]:
                batchsummary[field] = np.mean(batchsummary[field])
            print(batchsummary)
            with open(os.path.join(bpath, 'log.csv'), 'a', newline='') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writerow(batchsummary)
                # deep copy the model if the jaccard is the best
                if phase == 'Val' and batchsummary['Val_jaccard_score'] > best_jaccard:
                    best_jaccard = batchsummary['Val_jaccard_score']
                    best_model_wts = copy.deepcopy(model.state_dict())
                    torch.save({'state_dict':best_model_wts}, os.path.join(bpath, model_name+'_best.pth.tar'))

            lr_scheduler.step()
            # Save the state  
            state = {'epoch': epoch, 'state_dict': model.state_dict(),
                      'optimizer': optimizer.state_dict(),
                      'scheduler':lr_scheduler.state_dict(), 
                      'best_jaccard': best_jaccard, }
            torch.save(state, os.path.join(bpath, model_name+'-epoch{}.pth'.format(epoch)))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Highest Jaccard: {:4f}'.format(best_jaccard))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

En el caso de la segmentación binaria, la salida de la red es una imagen con dos canales: uno para el *background* y otro para la clase a segmentar. El criterio para el entrenamiento de la red es la función `BCEWithLogitsLoss` para clasificación binaria.

El entrenamiento de la red se realiza durante 12 epochs, reduciendo la tasa de entrenamiento a medida que se avanza en el entrenamiento. El código produce un archivo denominado _log.csv_ donde se puede analizar la variación de las funciones de pérdida en cada _epoch_ de entrenamiento, así como la correspondiente precisión y recall en el conjunto de test tanto para detección como para clasificación. Compruebe que las funciones de pérdida son algo ruidosas, y que las medidas de evaluación (área bajo la curva ROC e índice Jaccard) en validación van creciendo a medida que avanza el entrenamiento.

__**IMPORTANTE:__ note cómo el área ROC no es descriptiva de cómo avanza el proceso de entrenamiento para nuestro caso (desde el principio es muy elevada, por encima del 95%) y no está correlada con el índice Jaccard. Esto ocurre por el __desbalanceo__ de la base de datos (la cúpula óptica representa una parte muy pequeña de las imágenes). En bases de datos muy desbalanceadas la curva ROC no es un buen indicativo de la eficiencia de la segmentación ya que la importancia que da a las clases depende de su probabilidad de aparición.

In [ ]:
# custom weight initialization 
def weights_init(m):
    if isinstance(m, torch.nn.Conv2d):
        torch.nn.init.xavier_normal_(m.weight,1.0)
        
model_name='deeplabv3_binary'

# Training stage
model.train()

# Create the experiment directory if not present
if not os.path.isdir(result_dir):
    os.mkdir(result_dir)

# Specify the loss function
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean') # 2 classes

# Specify the optimizer with a lower learning rate for backbone
params_classifier = [p for p in model.classifier.parameters() if p.requires_grad]

params_backbone = [p for p in model.backbone.parameters() if p.requires_grad]

# Initialize the classifier-conv_layer weights, to adapt to the new paradigm (retinography vs natural images)
torch.manual_seed(manualSeed)
model.classifier.apply(weights_init)

# We apply a different lr to backbone and classifier parts
optimizer = torch.optim.Adam([
{'params': params_backbone},
{'params': params_classifier, 'lr': 1e-3}
], lr=1e-4)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                          step_size=step_size,
                                          gamma=0.1)


# Specify the evaluation metrics
metrics = {'jaccard_score': jaccard_score, 'auroc': roc_auc_score}

trained_model = train_model(model, criterion, dataloaders, device,
                        optimizer, lr_scheduler, bpath=result_dir, model_name=model_name,
                        metrics=metrics, num_classes=num_classes_binary, num_epochs=epochs)


### Evaluación

Tras entrenar la red, se van a evaluar los resultados para el conjunto de test. En primer lugar, se cargan los pesos de la red entrenada en el modelo y se llama a la función de evaluación. Preste atención a los parámetros que recibe la función.

In [ ]:
# Inference
dataloaders = get_dataloader_riga(
    data_dir, maskFolder='masks', binary=True, use_only_train=use_only_train, batch_size=batchsize_test, img_size=img_size, data_augm=False, image_mean=mean,image_std=std)
weights=torch.load(os.path.join(result_dir,model_name+'_best.pth.tar'))['state_dict']
model = get_deeplabv3(num_classes_binary)
model.to(device)
model.load_state_dict(weights)
model.eval()
test_segmentation_model_binary(model, dataloaders, class_names_binary, 0.5, result_dir, True, batchsize_test)

## Parte 3. Experimentos.

### 1. Influencia del umbral en la inferencia.

Analice la influencia del umbrales TH para los resultados en inferencia para el problema de segmentación binaria, tanto visualmente (para ello ponga el parámetro `SAVE_OPT` a `True`) como con las medidas de segmentación. A la vista de los resultados, ¿ha aprendido la red a segmentar de manera coherente las imágenes?

In [ ]:
test_segmentation_model_binary(model, dataloaders, class_names_binary, 0.2, result_dir, False, batchsize_test)
test_segmentation_model_binary(model, dataloaders, class_names_binary, 0.8, result_dir, False, batchsize_test)

### 2. Utilidad de las capas *atrous* y el ASPP.

En este experimento, para poder observar la capacidad de representación del ASPP con capas *atrous*, se construye una red similar a Deeplab-V2 sobre la arquitectura que tenemos. En Deeplab-V2 las capas *atrous* se encuentran justamente por delante de la función de *loss*, de manera que sus activaciones están __directamente relacionadas__ con la segmentación de salida de la red. En nuestra red sin embargo, las capas *atrous* están __más alejadas__ de la función de *loss* de manera que es __más difícil analizar sus activaciones__. Para convertir nuestra arquitectura en la de Deeplab-V2, se realiza lo siguiente:

- Se elimina el bloque ASPP de Deeplab-V3, que contiene el *avg pooling*.
- Se añade el bloque ASPP de Deeplab-V2, con 4 convoluciones *atrous* de tasa $r=6, 12, 18$ y $24$. A las salidas de cada rama del ASPP se coloca una capa de convolución que convierte cada mapa de características al tamaño del mapa de salida (según el número de clases, en nuestro caso 2, *background* + cúpula óptica). 
- Se suman las contribuciones de cada rama para generar la salida y se aplica la función de *loss*.

La siguiente figura muestra los cambios que se realizan para visualizar las activaciones.

<img src="https://tsc.uc3m.es/~mmolina/images_segmentation/deeplabv3v2.PNG" width="600pix">

De este modo, se pueden visualizar las salidas marginales de cada rama y ver cómo funcionan las convoluciones *atrous* a partir de sus activaciones.

- Analice las activaciones de cada rama para las imágenes de test. ¿Qué diferencias encuentra y a qué se deben? ¿Cómo contribuyen las convoluciones *atrous* y el ASPP a mejorar la segmentación? ¿En qué caso cree que serán más útiles, cuando la segmentación sea densa (existen muchos objetos en la imagen), o en un caso como este?

In [ ]:
class Sum(torch.nn.Module):

    def __init__(self, num_classes):
        super(Sum, self).__init__()
        self.num_classes = num_classes
    def forward(self, x):
        x=x.unsqueeze(1)
        x=x.view(x.size(0),4,self.num_classes,x.size(3),x.size(4))
        x=torch.sum(x,dim=1)
        return x

def get_deeplabv2(num_classes=1):
    model=get_deeplabv3(num_classes=2)
    # Remove the last avg pooling
    model.classifier[0].convs=model.classifier[0].convs[:-1]
    # Change the convolutions from the remaining blocks r=6,12,28 and 24 and adding conv-layers to each branch
    modules = []
    modules.append(torch.nn.Conv2d(2048,256, kernel_size=3,stride=1,padding=6,dilation=6))
    modules.append(model.classifier[0].convs[0][1])
    modules.append(model.classifier[0].convs[0][2])
    modules.append(torch.nn.Conv2d(256, num_classes, kernel_size=1))
    model.classifier[0].convs[0]=torch.nn.Sequential(*modules)
    modules = []
    modules.append(torch.nn.Conv2d(2048,256, kernel_size=3,stride=1,padding=12,dilation=12))
    modules.append(model.classifier[0].convs[1][1])
    modules.append(model.classifier[0].convs[1][2])
    modules.append(torch.nn.Conv2d(256, num_classes, kernel_size=1))
    model.classifier[0].convs[1]=torch.nn.Sequential(*modules)
    modules = []
    modules.append(torch.nn.Conv2d(2048,256, kernel_size=3,stride=1,padding=18,dilation=18))
    modules.append(model.classifier[0].convs[2][1])
    modules.append(model.classifier[0].convs[2][2])
    modules.append(torch.nn.Conv2d(256, num_classes, kernel_size=1))
    model.classifier[0].convs[2]=torch.nn.Sequential(*modules)
    modules = []
    modules.append(torch.nn.Conv2d(2048,256, kernel_size=3,stride=1,padding=24,dilation=24))
    modules.append(model.classifier[0].convs[3][1])
    modules.append(model.classifier[0].convs[3][2])
    modules.append(torch.nn.Conv2d(256, num_classes, kernel_size=1))
    model.classifier[0].convs[3]=torch.nn.Sequential(*modules)
    # Sum the marginal predictions for each branch
    model.classifier[0].project=Sum(num_classes_binary)
    model.classifier=model.classifier[0]
    return model

model_name='deeplabv2'

dataloaders = get_dataloader_riga(
    data_dir, maskFolder='masks', binary=True, use_only_train=use_only_train, batch_size=batchsize_train, img_size=img_size, data_augm=False, image_mean=mean,image_std=std)
model = get_deeplabv2(num_classes_binary)
# Use gpu if available
model.to(device)
model.train()

params_classifier = [p for p in model.classifier.parameters() if p.requires_grad]

params_backbone = [p for p in model.backbone.parameters() if p.requires_grad]

torch.manual_seed(manualSeed)
model.classifier.apply(weights_init)


# Specify the loss function
criterion = torch.nn.BCEWithLogitsLoss(reduction='mean') # 2 classes

# Specify the optimizer with a lower learning rate
optimizer = torch.optim.Adam([
{'params': params_backbone},
{'params': params_classifier, 'lr': 1e-3}
], lr=1e-4)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                          step_size=step_size,
                                          gamma=0.1)

# Specify the evaluation metrics
metrics = {'jaccard_score': jaccard_score, 'auroc': roc_auc_score}

trained_model = train_model(model, criterion, dataloaders, device,
                        optimizer, lr_scheduler, bpath=result_dir, model_name=model_name,
                        metrics=metrics, num_classes=num_classes_binary, num_epochs=epochs)


__**IMPORTANTE:__ la función externa `get_activations` proporciona una representación visual de las salidas del ASPP, que almacena en la carpeta `activations` en el directorio de resultados. Sus entradas son:

- __model__: la CNN que evaluar.
- __dataloaders__: el cargador de los datos de test.
- __device__: el dispositivo que utilizar para el entrenamiento (GPU o CPU).
- __result_dir__: el directorio de resultados.
- __batchsize_test__: el tamaño de batch para test, que debe ser 1.

__**IMPORTANTE:__ la función `get_activations` hace uso de los *hook* de Pytorch. Los *hook* de Pytorch son una serie de funciones que permiten modificar los datos de entrada o salida de alguna capa de la red durante la ejecución de la red. Esto permite tener acceso a datos intermedios de la red en tiempo de ejecución, así como realizar un *debug* controlado de la red neuronal. En concreto se usa un *register_forward_hook()*, se ejecuta tras el método *forward* de cualquier capa de la red y tiene acceso a sus entradas y salidas. 

In [ ]:
from external import get_activations
# Code to obtain the activations
dataloaders = get_dataloader_riga(
data_dir, maskFolder='masks', binary=True, use_only_train=use_only_train, batch_size=batchsize_test, img_size=img_size, data_augm=False, image_mean=mean,image_std=std)
weights=torch.load(os.path.join(result_dir,model_name+'_best.pth.tar'))['state_dict']
model = get_deeplabv2(num_classes_binary)
model.to(device)
model.load_state_dict(weights)
model.eval()
get_activations(model, dataloaders, device, result_dir, batchsize_test)

### 3. Segmentación multi-clase. 

En este apartado se va a abordar la segmentación multi-clase para la tarea que se propone, con las clases cúpula óptica y disco óptico. En primer lugar, se modifica la función de testeo de la segmentación, que ya no admite umbralizar los resultados (en el caso de la segmentación multi-clase es el valor máximo del `softmax` el que proporciona la clase predicha). En cualquier caso, se podrían aplicar pesos al entrenamiento para primar unas clases sobre otras. La función almacena los resultados de las predicciones en el directorio `predictions_multiclass`

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def test_segmentation_model_multiclass(model, dataloaders, num_classes, class_names, result_dir, SAVE_OPT, batch_size=1):
    cm_total=np.zeros(len(class_names))
    # Evaluation: Jaccard Index
    jaccard=[]
    # We create the results folder and CSV file for results if they do not exist
    if not os.path.exists(result_dir):
        os.mkdir(result_dir)
    else:
        if os.path.exists(os.path.join(result_dir,'results_multiclass.csv')):
            os.remove(os.path.join(result_dir,'results_multiclass.csv'))
    if (SAVE_OPT):
        # We create the folder for predictions
        if not os.path.exists(os.path.join(result_dir,'predictions_multiclass')):
            os.mkdir(os.path.join(result_dir,'predictions_multiclass'))
    csv_file=open(os.path.join(result_dir,'results_multiclass.csv'),'w')
    coord_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for sample in dataloaders['Test']:
        with torch.no_grad():
            inputs = sample['image'].to(device)
            masks = sample['mask'].to(device)
            prediction = model(inputs)
            y_pred = prediction['out']
            y_pred = torch.nn.functional.softmax(y_pred, dim=1)
            _,y_pred = torch.max(y_pred, dim=1)
            y_pred = y_pred.data.cpu().numpy()
            y_true = masks.data.cpu().numpy()
            for j in range(y_pred.shape[0]):
                img_name=sample['img_path'][j].split(os.path.sep)[-1]
                dataset_name=sample['img_path'][j].split(os.path.sep)[-2]
                ji=np.zeros((num_classes-1,),dtype='float')
                for i in range(num_classes-1):
                    # We measure with the Jaccard Index
                    ji[i]=np.sum(np.logical_and(np.squeeze(y_pred[j,:,:]==(i+1)),np.squeeze(y_true==(i+1)))/(np.sum(y_pred[j,:,:]==(i+1))+np.sum(y_true[j,:,:]==(i+1))-np.sum(np.logical_and(np.squeeze(y_pred[j,:,:]==(i+1)),np.squeeze(y_true[j,:,:]==(i+1))))))
                jaccard.append(ji)
                cm_total=cm_total+confusion_matrix(y_true[j,:,:].reshape(-1,1), y_pred[j,:,:].reshape(-1,1), labels=None, sample_weight=None, normalize=None)
                if (SAVE_OPT):
                    img=Image.open(sample['img_path'][j]).resize((img_size,img_size))
                    mask=(np.transpose(np.concatenate(((y_pred[j,:,:])[np.newaxis,:,:],y_true[j,:,:][np.newaxis,:,:],np.zeros_like(y_pred[j,:,:][np.newaxis,:,:])),axis=0),(1,2,0))*255.0/num_classes).astype(np.uint8)
                    img = Image.blend(img.convert('RGBA'), Image.fromarray(mask).convert('RGBA'),0.5)
                    img.save(os.path.join(result_dir,'predictions_multiclass',img_name[:-4]+'.png'))
                coord_writer.writerow([img_name,str(ji)])
    # Mean values
    coord_writer.writerow(['MEAN',str(sum(jaccard)/len(jaccard))])
    print('Jaccard index for segmentation: {}'.format(sum(jaccard)/len(jaccard)))
    csv_file.close()
    return cm_total

In [ ]:
model = get_deeplabv3(num_classes_multiclass)
# Use gpu if available
model.to(device)

model_name='deeplabv3_multiclass'

# Multi-class database
dataloaders = get_dataloader_riga(
    data_dir, maskFolder='masks_full', binary=False, use_only_train=use_only_train, batch_size=batchsize_train, img_size=img_size, data_augm=False, image_mean=mean,image_std=std)

# Training stage
model.train()

# Create the experiment directory if not present
if not os.path.isdir(result_dir):
    os.mkdir(result_dir)

# Specify the loss function
criterion = torch.nn.CrossEntropyLoss(reduction='mean')# 3 classes

# Specify the optimizer with a lower learning rate for backbone
params_classifier = [p for p in model.classifier.parameters() if p.requires_grad]

params_backbone = [p for p in model.backbone.parameters() if p.requires_grad]

# Initialize the classifier-conv_layer weights, to adapt to the new paradigm (retinography vs natural images)
torch.manual_seed(manualSeed)
model.classifier.apply(weights_init)

# We apply a different lr to backbone and classifier parts
optimizer = torch.optim.Adam([
{'params': params_backbone},
{'params': params_classifier, 'lr': 1e-3}
], lr=1e-4)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                          step_size=step_size,
                                          gamma=0.1)


# Specify the evaluation metrics
metrics = {'jaccard_score': jaccard_score, 'auroc': roc_auc_score}

trained_model = train_model(model, criterion, dataloaders, device,
                        optimizer, lr_scheduler, bpath=result_dir, model_name=model_name,
                        metrics=metrics, num_classes=num_classes_multiclass, num_epochs=epochs)


Efectúe la inferencia para el modelo y analice los resultados. 

- ¿Qué ocurre al introducir la nueva clase sobre el índice Jaccard de la cúpula óptica?
- Analice la matriz de confusión. ¿Qué clases es más sencillo confundir entre sí? Justifíquelo debido a la topología de las clases. 

In [ ]:
# Inference
dataloaders = get_dataloader_riga(
    data_dir, maskFolder='masks_full', binary=False, use_only_train=use_only_train, batch_size=batchsize_test, img_size=img_size, data_augm=False, image_mean=mean,image_std=std)
weights=torch.load(os.path.join(result_dir,model_name+'_best.pth.tar'))['state_dict']
model = get_deeplabv3(num_classes_multiclass)
model.load_state_dict(weights)
model.to(device)
model.eval()
cm=test_segmentation_model_multiclass(model, dataloaders, num_classes_multiclass, class_names_multiclass, result_dir, True, batchsize_test)
dconf=ConfusionMatrixDisplay(cm,display_labels=class_names_multiclass)
dconf.plot()

### 4. Funciones de pérdida.

En este apartado se van a describir situaciones  con la función de pérdida de la red para posteriores experimentos. A la hora de entrenar una red de segmentación, hay que tener en cuenta dos cosas principalmente: el desbalanceo de las clases; si existe, y el uso de estrategias de regularización de la función de pérdida pixelar. Ambas se definen a continuación.

#### Desbalanceo de las clases

El desbalanceo de las clases ocurre cuando algunas de las clases en la salida dominan sobre el resto (es decir, la proporción de píxeles pertenecientes a las distintas clases es muy desigual). Esto puede provocar que la red tienda a dar demasiada importancia a las clases más representadas a costa de reducir la importancia (o incluso hacer desaparecer) clases poco representadas. Esto se puede comprender mejor si se analiza a nivel de píxel: __la función de pérdida estándar aplica la misma importancia a todos los píxeles de la salida, independientemente de su clase, es decir, la red se va a centrar en clasificar cada uno de ellos correctamente, sea cual sea su clase. Sin embargo, si existe desbalanceo para las clases, un error en una clase poco representada (1 error sobre 10 píxeles, por ejemplo) será más perjudicial para el rendimiento del sistema que un error en una clase muy representada (1/1000)__. Si se une esto a que la red puede aprender mejor la clase muy representada porque tiene un mayor número de ejemplos de la misma, el resultado de la segmentación puede ser poco preciso

Como estrategia para mitigar esto se propone una muy sencilla: se basa en aplicar una serie de pesos a las clases en la función de pérdida según su probabilidad de aparición en el conjunto de entrenamiento. Esto provoca que la red no ponga el mismo énfasis en clasificar cada píxel, sino que un error en un píxel de una clase poco representada dará un valor de la función de pérdida mayor que en una clase muy representada, de manera que la red pone más énfasis en resolver los errores sobre la clase más "difícil". Sin embargo, esta estrategia es sensible a los pesos que se apliquen a las clases. 

#### Estrategias de regularización

Por otra parte, la función de pérdida pixelar no tiene en cuenta ninguna dependencia entre los píxeles de la imagen (es decir, a la función contribuyen de igual manera todos los píxeles de la imagen; y entre ellos no se impone ninguna relación). Sin embargo, en el caso de la segmentación de imágenes existe una fuerte dependencia entre los píxeles:

- A nivel local, píxeles adyacentes a uno clasificado como perteneciente a la clase 'X' por ejemplo, tienen más probabilidad de ser de la clase 'X' que de la clase 'Y' en imágenes naturales (los objetos son continuos hasta que se llega a sus bordes). Existen estrategias sobre la función de pérdida que pueden reforzar la coherencia local en las segmentaciones (véase *'pairwise penalties'* en [6]).
- A nivel de imagen, existirán imágenes que la red segmente correctamente y otras en las que el desempeño sea menor. Existen funciones de pérdida que modelan esto (hacen que la red se centre en las imágenes más complejas) y combinadas con la función pixelar pueden resultar de ayuda, como la Dice Loss, en [8].
- En objetos no uniformes (con regiones con aspectos muy diferentes), pueden existir ciertos tipos de regiones que la red segmente muy bien y otras que sean complejas. Una función de pérdida que dé mayor importancia a los píxeles mal clasificados que a los correctamente clasificados puede ser útil. De este modo la red se centrará en clasificar correctamente dichos píxeles (lo que frecuentemente no afecta a los píxeles ya correctamente clasificados, que son más sencillos) y los resultados mejorarán. Esto equivale a hacer un procedimiento de *Hard Negative Mining* sobre los píxeles (es decir, centrarse en aquellos que resultan más difíciles para la red). La función Focal Loss [7] puede ser un buen ejemplo. 

### 4. Evaluación del trabajo autónomo del alumno.

#### Criterios de evaluación

De esta práctica (si elegida) surge la segunda evaluación para la asignatura. Una vez comprendidos los fundamentos de la red de segmentación, puede realizar los experimentos que considere oportunos. Estos experimentos pueden ir dirigidos a:

- Profundizar en la arquitectura (observar la dependencia de los resultados con las modificaciones de la misma, especialmente en la parte del clasificador: convoluciones *atrous*).
- Analizar los resultados y proporcionar estrategias de mejora.
- Modificar el proceso de entrenamiento de la red a través de la función de pérdida, con las estrategias descritas en el apartado anterior u otras.


#### Entregables

- Presentación (Fecha indicada en la entrega del proyecto en Aula Global). Este día cada grupo de alumnos tendrá un turno de 10 minutos de preguntas (máximo 5 minutos de presentación) sobre el apartado de trabajo autónomo con ayuda de un máximo de 3 transparencias.
- Informe + Código. Los alumnos entregarán un breve informe (2 caras para la descripción, 1 cara de referencias y figuras si fuese necesaria) donde describirán los aspectos más importantes de la solución propuesta. El objetivo es que el alumno describa los análisis y extensiones que ha planteado al modelo y justifique su objetivo y utilidad de manera breve. Asimismo, se proporcionará el código utilizado para los experimentos (bien sobre este mismo Notebook, en formato `.ipynb` o bien en código Python, en formato `.py`). 

La fecha límite de entrega del fichero de código y el informe es la fecha indicada en la entrega del proyecto en Aula Global.

#### Sugerencias

A continuación se proporcionan algunas sugerencias para que el alumno trabaje de manera autónoma, a título informativo. Si lo desea, puede centrarse en implementar una o varias de ellas.

- Se puede trabajar desde el punto de vista del *data augmentation*, efectuando un procedimiento __adecuado a alguna de las tareas de segmentación__ para ampliar la variabilidad de la base de datos.

- Se puede modificar la estructura del clasificador de la red para adecuarlo a la tarea de segmentación con la que se trabaja (muestre especial atención al *receptive field* de la red). Además, se puede utilizar el bloque de clasificación auxiliar de la red (`aux_classifier`), modificando su estructura y añadiendo una función de pérdida para el caso de segmentación multi-clase. 

- A partir de las ideas proporcionadas en el apartado de funciones de pérdida, puede implementar alguna de las estrategias de regularización que se han descrito u otras que considere oportunas. Justifique su utilidad para la tarea que se propone.

- Si lo desea, utilice otras bases de datos para segmentación binaria o multi-clase e implemente estrategias para mejorar los resultados en la tarea de segmentación de objetos en las mismas.

__**IMPORTANTE:__ si encuentra problemas para entrenar la red Deeplab-V3 con backbone ResNet-101 o el entrenamiento es demasiado lento y desea agilizar los experimentos, el siguiente fragmento de código implementa Deeplab-V3 con un backbone mucho más ligero, de ResNet-18. Puede utilizarla como red *baseline* para el apartado autónomo. Tenga en cuenta que al no ser una red pre-entrenada en una base de datos para segmentación los resultados serán algo peores, especialmente en las activaciones *atrous* de la red, que visualmente no serán tan claras como en la red tratada.

In [ ]:
from torchvision.models._utils import IntermediateLayerGetter
from torchvision.models.segmentation.fcn import FCNHead
from torchvision.models import resnet18

def get_deeplabv3(num_classes=1):
    model = deeplabv3_resnet101(pretrained=True, progress=True)
    backbone = resnet18(pretrained=True)
    return_layers = {'layer4': 'out'}
    return_layers['layer3'] = 'aux'
    model.backbone = IntermediateLayerGetter(backbone, return_layers=return_layers)
    # Replace stride with dilation in blocks 3 and 4
    model.backbone.layer3[0].conv1.stride=(1,1)
    model.backbone.layer4[0].conv1.stride=(1,1)
    model.backbone.layer3[0].conv1.dilation=(2,2)
    model.backbone.layer3[0].conv1.padding=(2,2)
    model.backbone.layer4[0].conv1.dilation=(4,4)
    model.backbone.layer4[0].conv1.padding=(4,4)
    model.backbone.layer3[0].downsample[0].stride=(1,1)
    model.backbone.layer4[0].downsample[0].stride=(1,1)
    
    model.classifier = DeepLabHead(512, num_classes)
    model.aux_classifier=FCNHead(256,num_classes)
    return model


__**IMPORTANTE:__ es posible que la base de datos que se proporciona no constituya un buen *baseline* para los experimentos del apartado autónomo (la tarea de segmentación es bastante sencilla). Adicionalmente, se proporciona una base de datos de microscopía para una tarea más compleja, segmentación binaria de neutrófilos en vasos sanguíneos (en la carpeta __neutrophils__) o puede utilizar otras bases de datos disponibles.